# 데이터 수집.

In [1]:
from selenium import webdriver
from pandas.io.html import read_html
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

# 1.selenium을 이용한 데이터 수집

In [2]:
browser=webdriver.Chrome()
browser.maximize_window()
sleep_time=3
page_num=30
# 후스코드어 홈페이지 접속.
url="https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/PlayerStatistics/England-Premier-League-2020-2021"
browser.get(url)

In [3]:
#df.columns 생성 (후드 스코어의 수비,공격,패스 카테고리별로 기록)
#해보니 player가 두개 생성 되므로, 둘다 받아온 후 처리 할 예정.
df_Passing=pd.DataFrame(columns=['Player', 'Player.1', 'Apps', 'Mins','Assists','KeyP','AvgP','PS%','Crosses','LongB','ThrB','Rating'])
df_Defensive=pd.DataFrame(columns=['Player', 'Player.1', 'Apps', 'Mins','Tackles','Inter','Fouls','Offsides','Clear','Drb','Blocks','OwnG','Rating'])
df_Offensive = pd.DataFrame(columns=['Player', 'Player.1', 'Apps', 'Mins', 'Goals', 'Assists', 'SpG','KeyP', 'Drb', 'Fouled', 'Off','Disp','UnsTch','Rating'])

In [4]:
#Defensive 클릭
time.sleep(sleep_time)
Defensive=browser.find_element_by_xpath('//*[@id="stage-top-player-stats-options"]/li[2]/a')
Defensive.click()

In [5]:
#DataFrame에 데이터 입력.
for i in np.arange(page_num)+1:
    time.sleep(sleep_time)
    table= browser.find_element_by_xpath('//*[@id="statistics-table-defensive"]')
    table_html= table.get_attribute('innerHTML')
    df2=read_html(table_html)[0]
    df_Defensive=pd.concat([df_Defensive,df2],axis=0)
    browser.find_element_by_link_text('next').click()

In [6]:
#Offensive 클릭
time.sleep(sleep_time)
Offensive=browser.find_element_by_xpath('//*[@id="stage-top-player-stats-options"]/li[3]/a')
Offensive.click()

In [7]:
#DataFrame에 데이터 입력.
for i in np.arange(page_num)+1:
    time.sleep(sleep_time)
    table= browser.find_element_by_xpath('//*[@id="statistics-table-offensive"]')
    table_html= table.get_attribute('innerHTML')
    df3=read_html(table_html)[0]
    df_Offensive=pd.concat([df_Offensive,df3],axis=0)
    browser.find_element_by_link_text('next').click()

In [8]:
#Passing 클릭
time.sleep(sleep_time)
Passing=browser.find_element_by_xpath('//*[@id="stage-top-player-stats-options"]/li[4]/a')
Passing.click()

In [9]:
#DataFrame에 데이터 입력.
for i in np.arange(page_num)+1:
    time.sleep(sleep_time)
    table= browser.find_element_by_xpath('//*[@id="statistics-table-passing"]')
    table_html= table.get_attribute('innerHTML')
    df3=read_html(table_html)[0]
    df_Passing=pd.concat([df_Passing,df3],axis=0)
    browser.find_element_by_link_text('next').click()

# 2.데이터 전처리 과정

In [10]:
# 컬럼 수정 함수.
def process(data):
    
    # 인덱스 수정
    data= data.reset_index()
    data.drop(['index','Player'],axis=1,inplace=True)
    spl=data['Player.1'].str.split(',')
    
    #1.이름
    name=[]
    for i in range(len(spl)):
        a = spl[i][0]
        name.append(a)
    data['name']=name
    
    #2.나이
    age=[]
    for i in range(len(spl)):
        a = int(spl[i][1])
        age.append(a)
    data['age']=age
    
    #3.포지션이 2개인 선수와 1개인 선수가 존재하니 구분해서 나눈다
    position1=[]
    for i in range(len(spl)):
        a = spl[i][2]
        position1.append(a)
    data['position1']=position1
    position2=[]
    for i in range(len(spl)):
        if len(spl[i])>3:
            a = spl[i][2]
        else:
            a = np.nan
        position2.append(a)
    data['position2']=position2
    
    #4.원래의 정보  삭제.
    data.drop('Player.1',axis=1,inplace=True)
    
    return data

In [11]:
Defensive=process(df_Defensive)
Offensive=process(df_Offensive)
Passing=process(df_Passing)

In [12]:
# '-' 값을 0으로 처리하기

In [13]:
for i in np.arange(len(Defensive.columns)):
    Defensive.iloc[Defensive.iloc[:,i]=='-',i]=0

In [14]:
for i in np.arange(len(Offensive.columns)):
    Offensive.iloc[Offensive.iloc[:,i]=='-',i]=0

In [15]:
for i in np.arange(len(Passing.columns)):
    Passing.iloc[Passing.iloc[:,i]=='-',i]=0

In [16]:
# 컬럼 순서 재배치.

In [17]:
col1=Defensive.columns[-4:].to_list()
col2=Defensive.columns[:-4].to_list()
new_col=col1+col2
Defensive=Defensive[new_col]

In [18]:
col1=Offensive.columns[-4:].to_list()
col2=Offensive.columns[:-4].to_list()
new_col=col1+col2
Offensive=Offensive[new_col]

In [19]:
col1=Passing.columns[-4:].to_list()
col2=Passing.columns[:-4].to_list()
new_col=col1+col2
Passing=Passing[new_col]

# 3.csv 파일로 저장하기.

In [20]:
Defensive.to_csv('EPL_2020-2021_Player_Defensive_data.csv',sep=',')
Offensive.to_csv('EPL_2020-2021_Player_Offensive_data.csv',sep=',')
Passing.to_csv('EPL_2020-2021_Player_Passing_data.csv',sep=',')

In [22]:
browser.close()